<a href="https://colab.research.google.com/github/wanghaiy2018/AC0499_2024/blob/main/Code/ai_agent/medicaLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import json
import re
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Custom class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url: str):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.2",
            "prompt": str(prompt),  # Ensure the prompt is a string
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["age", "medical_history", "current_condition", "family_preferences", "social_media_posts", "personal_messages"],
    template="""
    The patient data is as follows:
    Age: {age}
    Medical History: {medical_history}
    Current Condition: {current_condition}
    Family Preferences: {family_preferences}
    Social Media Posts: {social_media_posts}
    Personal Messages: {personal_messages}
    Given this information, what is the most appropriate end-of-life care decision for this patient?
    """
)

# Environment class for end-of-life care
class CareEnvironment:
    def __init__(self, patient_data):
        self.state = 'initial_state'
        self.goal_achieved = False
        self.result = None
        self.patient_data = patient_data

    def get_state(self):
        return self.state

    def change_state(self, result):
        self.state = 'goal_state'
        self.result = result

# ReActAgent class modified for end-of-life care
class EndOfLifeCareAgent:
    def __init__(self, environment, llm, prompt_template, max_attempts=5):
        self.environment = environment
        self.llm = llm
        self.prompt_template = prompt_template
        self.max_attempts = max_attempts

    def perceive(self):
        return self.environment.get_state()

    def reason(self, state):
        prompt = self.create_prompt(self.environment.patient_data)
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        decision = self.extract_decision(response)
        return 'achieve_goal' if decision else 'take_action', decision

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True
            self.environment.change_state(result)

    def create_prompt(self, patient_data):
        return self.prompt_template.format_prompt(
            age=patient_data['age'],
            medical_history=", ".join(patient_data['medical_history']),
            current_condition=patient_data['current_condition'],
            family_preferences=", ".join(patient_data['family_preferences']),
            social_media_posts=patient_data['social_media_posts'],
            personal_messages=patient_data['personal_messages']
        ).to_string()

    def extract_decision(self, response):
        # Extracts a decision from the Llama model's response
        decisions = ["palliative care", "comfort care", "aggressive treatment", "life support", "hospice"]
        for decision in decisions:
            if decision in response.lower():
                return decision
        return None

# Example patient data
patient_data = {
    "age": 75,
    "medical_history": ["stroke", "diabetes", "hypertension"],
    "current_condition": "unresponsive",
    "family_preferences": ["comfort care"],
    "social_media_posts": "I value quality of life over prolonged suffering.",
    "personal_messages": "I don't want to be kept alive artificially if there's no hope of recovery."
}

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Create environment and agent
env = CareEnvironment(patient_data)
agent = EndOfLifeCareAgent(env, llm, prompt_template)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")


Raw Llama Response: I can't provide medical advice. If you or someone you know is in need of end-of-life care, I recommend consulting with a qualified healthcare professional, such as an oncologist, palliative care specialist, or hospice nurse, who can provide guidance based on the individual's specific needs and circumstances. Is there anything else I can help you with?
State: initial_state, Action: achieve_goal, Result: palliative care, Attempts: 1
Goal achieved with result: palliative care
